In [45]:
!pip install SpeechRecognition
!pip install pyaudio
!pip install editdistance
!pip install gTTS
!pip install sklearn
!pip install nltk
# sudo apt install portaudio19-dev python3-pyaudio
# sudo apt-get install portaudio19-dev python-pyaudio python3-pyaudio
#!sudo apt update && sudo apt install espeak ffmpeg libespeak1 for offline text_to_voice convert

[sudo] password for hua: 


In [28]:
import pyttsx3
# initialize Text-to-speech engine
engine = pyttsx3.init()
# setting new voice rate (faster)
engine.setProperty("rate", 150)

# convert this text to speech

text = "Hello, Kondo"

engine.say(text)
# play the speech
engine.runAndWait()


In [29]:
import speech_recognition as sr


r = sr.Recognizer()


In [30]:
with sr.Microphone() as source:
    r.adjust_for_ambient_noise(source)
    data = r.record(source, duration=5)
    print("Sesinizi Tanımlıyor…")
    print(2)
    text = r.recognize_google(data,language='ru')
    print(1)
    print(text)


ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.acp.pcm.front.0:CARD=0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM front
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.acp.pcm.surround51.0:CARD=0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM surround21
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find defini

Sesinizi Tanımlıyor…
2


UnknownValueError: 

In [32]:
import io
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from gtts import gTTS
import os
warnings.filterwarnings('ignore')
import speech_recognition as sr 
import nltk
from nltk.stem import WordNetLemmatizer
#for downloading package files can be commented after First run
nltk.download('popular', quiet=True)
nltk.download('nps_chat',quiet=True)
nltk.download('punkt') 
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /home/hua/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hua/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]
# To Recognise input type as QUES. 
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features
featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [34]:
# Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [35]:
#Reading in the input_corpus
#with open('intro_join','r', encoding='utf8', errors ='ignore') as fin:
#     raw = fin.read().lower()
raw = text
#TOkenisation
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words
# Preprocessing
lemmer = WordNetLemmatizer()
def LemTokens(tokens): 
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [36]:
#colour palet
def prRed(skk): print("\033[91m {}\033[00m" .format(skk)) 
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk)) 
def prYellow(skk): print("\033[93m {}\033[00m" .format(skk)) 
def prLightPurple(skk): print("\033[94m {}\033[00m" .format(skk)) 
def prPurple(skk): print("\033[95m {}\033[00m" .format(skk)) 
def prCyan(skk): print("\033[96m {}\033[00m" .format(skk)) 
def prLightGray(skk): print("\033[97m {}\033[00m" .format(skk)) 
def prBlack(skk): print("\033[98m {}\033[00m" .format(skk))

In [37]:
# Generating response and processing 
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [38]:
#Recording voice input using microphone 
file = "file.mp3"
flag=True
fst="My name is Kondo. I will do what you want. If you want to exit, say Bye"
tts = gTTS(fst, lang="en",tld="com")
tts.save(file)
os.system("mpg123 " + file )
r = sr.Recognizer()
prYellow(fst)

 My name is Kondo. I will do what you want. If you want to exit, say Bye


sh: 1: mpg123: not found


In [1]:
# Yandex music API
!pip install yandex-music --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 659.1 kB/s eta 0:00:001m677.4 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 4.1 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 7.8 MB/s eta 0:00:00
  Created wheel for yandex-music: filename=yandex_music-2.0.1-py3-none-any.whl size=280220 sha256=4b849e15c836b5282109fe0e036e63a21159674a93fcf10bbb8fc5ae065e6543
  Stored in directory: /home/hua/.cache/pip/wheels/1c/f9/07/f0ba69082fae654c12359f7986adddda0aad858ef1f892cfc5
Successfully built yandex-music


In [5]:
from yandex_music import Client

client = Client("y0_AgAAAAAqvlDlAAiJcgAAAADSicCfF-W6WmPOSTaafBFgACMr9Eq5wU4").init()

In [65]:
client.tracks(['10994777:1193829', '40133452:5206873', '48966383:6693286', '51385674:7163467'])[0]
#client.tracks_download_info(38318)[0].download(filename="1.mpg")
client.search("Birds")["best"]["result"]["id"]#.download(filename="1.mpg")

44856687

In [67]:

from playsound import playsound
playsound("1.mpg")

#!pip install playsound
def play_music_by_id(id):
    client.tracks_download_info(38318)[0].download(filename="1.mpg") # add id
    playsound("1.mpg")
def play_music_by_name(name):
    client.tracks_download_info(client.search(name)["best"]["result"]["id"])[0].download(filename="1.mpg") # add id
    import time
    time.sleep(3)
    playsound("1.mpg")

play_music_by_name("Closed")


In [83]:
client.search("closed")["best"]["result"]["id"]

60362379

In [86]:
nothing  = lambda x:None
commands = {"Play music":lambda id:play_music_by_name(id), "Move Head":nothing , "Go forward":nothing, "Go backward":nothing, "Turn right":nothing, "Turn left":nothing}
key_words_command = {"music":commands["Play music"]}
key_words_command["music"](1)

Traceback (most recent call last):
  File "/home/hua/anaconda3/envs/kondo/lib/python3.10/site-packages/playsound.py", line 261, in <module>
    playsound(argv[1])
  File "/home/hua/anaconda3/envs/kondo/lib/python3.10/site-packages/playsound.py", line 189, in _playsoundNix
    bus.poll(Gst.MessageType.EOS, Gst.CLOCK_TIME_NONE)
KeyboardInterrupt


In [68]:
"music Tokyo Drift".find("music")

0

In [85]:
while(flag==True):
    with sr.Microphone() as source:
        audio= r.listen(source, phrase_time_limit=3)
    try:
        user_response = format(r.recognize_google(audio))
        print("\033[91m {}\033[00m" .format("YOU SAID : "+user_response))
    except sr.UnknownValueError:
        prYellow("Oops! Didn't catch that")
    
    #user_response = input()
    #user_response=user_response.lower()
    if ("play" in user_response):
        print("Start play music")
        key_words_command["music"](user_response.find("music") + len("music"))
    clas=classifier.classify(dialogue_act_features(user_response))
    print(clas)
    if(clas!='Bye'):
        if(clas=='Emotion'):
            flag=False
            text = "Kondo: You are welcome.."
            prYellow(text)
            engine.say(text)
            engine.runAndWait()
                
        else:
            if(greeting(user_response)!=None):

                text = greeting(user_response) 
                
                engine.say(text)
                engine.runAndWait()
                print("\033[93m {}\033[00m" .format("Kondo: "+ text))
            else:
                print("\033[93m {}\033[00m" .format("Kondo: ",end=""))
                res=(response(user_response))
                engine.say(res)
                engine.runAndWait()
                prYellow(res)
                sent_tokens.remove(user_response)
                tts = gTTS(res, 'en')
                tts.save(file)
                os.system("mpg123 " + file)
    else:
        flag=False
        text = "Kondo: Bye! take care.."
        prYellow(text)
        engine.say(text)
        # play the speech
        engine.runAndWait()

 YOU SAID : play claws
Start play music


KeyboardInterrupt: 